### Analyses of output files from the pipeline optimus v6.0.0+
##### Aim of this part of analysis is to detect doublets in the data using scDblFinder

NB: You need to install scDblFinder in a Unix environment

#### Load necessary libraries

In [1]:
from cellbender.remove_background.downstream import anndata_from_h5
import scanpy as sc

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(scater)
library(BiocParallel)
library(scDblFinder)
library(Seurat)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOr

WARNING: 
###### 1. The R package "reticulate" only fixed recently an issue that caused a segfault when used with rpy2:
###### https://github.com/rstudio/reticulate/pull/1188 : This issue is fixed in reticulate 1.25 (We use the version 1.24.0)
###### 2. the last release of r-seurat is 4.4.0, the installed version for this project

In [4]:
! R --version

R version 4.3.1 (2023-06-16) -- "Beagle Scouts"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-conda-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under the terms of the
GNU General Public License versions 2 or 3.
For more information about these matters see
https://www.gnu.org/licenses/.



In [5]:
%%R
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Debian GNU/Linux 11 (bullseye)

Matrix products: default
BLAS/LAPACK: /home/xiliu/.conda/envs/optimus/lib/libopenblasp-r0.3.24.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    tools     stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] SeuratObject_5.0.0          Seurat_4.4.0               
 [3] scDblFinder_1.14.0          BiocParallel_1.34.2        
 [5] scater_1.28.0               ggplot2_3.4.4              
 [7] scuttle_1.10.1              SingleCellExperiment_1.22.0
 [9] Summari

#### Using empty drop removal adata: according cellbender (filtered output)

cellbender output

In [20]:
adata = anndata_from_h5('/home/xiliu/terraPipelines/scAtlas/tmp/10k_pbmc_v3_out_filtered_cellbenderDefault.h5')

Assuming we are loading a "filtered" file that contains only cells.


/home/xiliu/.conda/envs/optimus/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


In [21]:
data_mat = adata.X.T

In [22]:
# Conversion 'py2rpy' not defined for objects of type '<class 'scipy.sparse._csc.csc_matrix'>'
data_dense_matrix = data_mat.toarray()

In [23]:
%%R -i data_dense_matrix -o doublet_score -o doublet_class

set.seed(123)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_dense_matrix),
    ) 
)
doublet_score = sce$scDblFinder.score
doublet_class = sce$scDblFinder.class

Creating ~10049 artificial doublets...
Dimensional reduction
Evaluating kNN...
Training model...
iter=0, 2376 cells excluded from training.
iter=1, 2215 cells excluded from training.
iter=2, 2107 cells excluded from training.
Threshold found:0.423
929 (7.4%) doublets called
In addition: Warning message:
In .checkSCE(sce) :
  Some cells in `sce` have an extremely low read counts; note that these could trigger errors and might best be filtered out


###### Stability check: seed = 10 => 1025 (8,2%) doublets, seed = 50 => 910 (7.2%) doublets

Add doublet detection results to AnnData object in .obs

In [24]:
adata.obs["scDblFinder_score"] = doublet_score
adata.obs["scDblFinder_class"] = doublet_class
adata.obs.scDblFinder_class.value_counts()

scDblFinder_class
singlet    11632
doublet      929
Name: count, dtype: int64

In [25]:
PATH_TO_OUTPUT_CELLBENDER = "/home/xiliu/terraPipelines/scAtlas/tmp/10k_pbmc_v3_out_filtered_cellbenderDefault_scDblFinder.h5ad"
adata.write_h5ad(PATH_TO_OUTPUT_CELLBENDER)

Using empty drop removal adata: EmptyDrop

In [4]:
input_h5ad = "/home/xiliu/terraPipelines/scAtlas/tmp/10k_pbmc_v3.h5ad"
adata_optimus = sc.read_h5ad(input_h5ad)

/home/xiliu/.conda/envs/optimus/lib/python3.9/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [5]:
adata_emptydrops = adata_optimus[adata_optimus.obs.emptydrops_IsCell, :]
print(adata_emptydrops.obs.shape[0])

11031


In [6]:
data_mat_emptydrops = adata_emptydrops.X.T
data_dense_matrix_emptydrops = data_mat_emptydrops.toarray()

In [7]:
%%R -i data_dense_matrix_emptydrops -o doublet_score_emptydrops -o doublet_class_emptydrops

set.seed(123)
sce_emptydrops = scDblFinder(
    SingleCellExperiment(
        list(counts=data_dense_matrix_emptydrops),
    ) 
)
doublet_score_emptydrops = sce_emptydrops$scDblFinder.score
doublet_class_emptydrops = sce_emptydrops$scDblFinder.class

Creating ~8825 artificial doublets...
Dimensional reduction
Evaluating kNN...
Training model...
iter=0, 1651 cells excluded from training.
iter=1, 1608 cells excluded from training.
iter=2, 1534 cells excluded from training.
Threshold found:0.366
927 (8.4%) doublets called
In addition: Warning message:
In .checkSCE(sce) :
  Some cells in `sce` have an extremely low read counts; note that these could trigger errors and might best be filtered out


In [8]:
adata_emptydrops.obs["scDblFinder_score"] = doublet_score_emptydrops
adata_emptydrops.obs["scDblFinder_class"] = doublet_class_emptydrops
adata_emptydrops.obs.scDblFinder_class.value_counts()

/tmp/ipykernel_20078/2921475152.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_emptydrops.obs["scDblFinder_score"] = doublet_score_emptydrops
/home/xiliu/.conda/envs/optimus/lib/python3.9/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


scDblFinder_class
singlet    10104
doublet      927
Name: count, dtype: int64

In [9]:
PATH_TO_OUTPUT_EMPTYDROP = "/home/xiliu/terraPipelines/scAtlas/tmp/10k_pbmc_v3_emptydrop_scDblFinder.h5ad"
adata_emptydrops.write_h5ad(PATH_TO_OUTPUT_EMPTYDROP)

#### Overlap between doublets detected using CellBender and EmptyDrop's empty droplets removal

In [26]:
adata_cellbender = sc.read_h5ad(PATH_TO_OUTPUT_CELLBENDER)
cellbender_BC = adata_cellbender.obs.index.to_list()
cellbender_BC_doublet = adata_cellbender.obs[adata_cellbender.obs["scDblFinder_class"] == "doublet"].index.to_list()
print(len(cellbender_BC))
print(len(cellbender_BC_doublet))

12561
929


/home/xiliu/.conda/envs/optimus/lib/python3.9/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [30]:
adata_emptydrops = sc.read_h5ad(PATH_TO_OUTPUT_EMPTYDROP)
emptydrops_BC = adata_emptydrops.obs.index.to_list()
emptydrops_BC_doublet = adata_emptydrops.obs[adata_emptydrops.obs["scDblFinder_class"] == "doublet"].index.to_list()
print(len(emptydrops_BC))
print(len(emptydrops_BC_doublet))

11031
927


/home/xiliu/.conda/envs/optimus/lib/python3.9/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [28]:
# Common barcodes between cellbender and emptyDrops cell calling
cellbender_emptydrops_BC = set(cellbender_BC).intersection(emptydrops_BC)
print(len(cellbender_emptydrops_BC))

10772


In [29]:
# doublet overlapping of the common barcodes
common_cellbender_BC_doublet = [i for i in cellbender_BC_doublet if i in cellbender_emptydrops_BC]
common_emptydrops_BC_doublet = [i for i in emptydrops_BC_doublet if i in cellbender_emptydrops_BC]
common_doublet = set(common_cellbender_BC_doublet).intersection(common_emptydrops_BC_doublet)
print(f'the common doublets identified from both cellbender and emptydrops cell calling: {len(common_doublet)}')

the common doublets identified from both cellbender and emptydrops cell calling: 815
